## 1. Project Overview & Objectives

In [48]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import os
import re
import gc
import warnings

# Visualization settings
plt.style.use("seaborn-v0_8")
sns.set()
warnings.filterwarnings("ignore")

# Display settings
pd.set_option("display.max_columns", None)
pd.set_option("display.float_format", lambda x: f"{x:,.4f}")


### 2. Load Data 

In [49]:
import pandas as pd
import os

# Path relative to notebook location:
DATA_PATH = "../data/raw"

def load_olist_data():
    datasets = {
        "customers": "olist_customers_dataset.csv",
        "geolocation": "olist_geolocation_dataset.csv",
        "order_items": "olist_order_items_dataset.csv",
        "order_payments": "olist_order_payments_dataset.csv",
        "order_reviews": "olist_order_reviews_dataset.csv",
        "orders": "olist_orders_dataset.csv",
        "products": "olist_products_dataset.csv",
        "sellers": "olist_sellers_dataset.csv",
        "category_translation": "product_category_name_translation.csv",
    }
    
    loaded = {}
    for name, filename in datasets.items():
        path = os.path.join(DATA_PATH, filename)

        if filename.endswith(".tsv"):
            df = pd.read_csv(path, sep="\t", encoding="latin1", low_memory=False)
        else:
            df = pd.read_csv(path, encoding="latin1", low_memory=False)

        loaded[name] = df
        print(f"Loaded: {name} → {df.shape}")

    return loaded

# Load everything
data = load_olist_data()

Loaded: customers → (99441, 5)
Loaded: geolocation → (1000163, 5)
Loaded: order_items → (112650, 7)
Loaded: order_payments → (103886, 5)
Loaded: order_reviews → (99224, 7)
Loaded: orders → (99441, 8)
Loaded: products → (32951, 9)
Loaded: sellers → (3095, 4)
Loaded: category_translation → (71, 2)


In [50]:
# Build a Single Master Table
data = load_olist_data()

customers            = data["customers"]
geolocation          = data["geolocation"]
order_items          = data["order_items"]
order_payments       = data["order_payments"]
order_reviews        = data["order_reviews"]
orders               = data["orders"]
products             = data["products"]
sellers              = data["sellers"]
category_translation = data["category_translation"]

Loaded: customers → (99441, 5)
Loaded: geolocation → (1000163, 5)
Loaded: order_items → (112650, 7)
Loaded: order_payments → (103886, 5)
Loaded: order_reviews → (99224, 7)
Loaded: orders → (99441, 8)
Loaded: products → (32951, 9)
Loaded: sellers → (3095, 4)
Loaded: category_translation → (71, 2)


In [51]:
orders.head()
products.info()
order_items.sample(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32951 entries, 0 to 32950
Data columns (total 9 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   product_id                  32951 non-null  object 
 1   product_category_name       32341 non-null  object 
 2   product_name_lenght         32341 non-null  float64
 3   product_description_lenght  32341 non-null  float64
 4   product_photos_qty          32341 non-null  float64
 5   product_weight_g            32949 non-null  float64
 6   product_length_cm           32949 non-null  float64
 7   product_height_cm           32949 non-null  float64
 8   product_width_cm            32949 non-null  float64
dtypes: float64(7), object(2)
memory usage: 2.3+ MB


,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
100621,e42760f40f2fba3ba7529b6157adf501,1,9e4a852f41aff52bafff133d9deb5eee,3d871de0142ce09b7081e2b9d1733cb1,2018-01-16 16:27:49,129.0000,18.0000
64921,945552359381c20f8e692b034ba74c44,1,5d10b3779b68d58be0fcc46a1ee79689,45d33f715e24d15a6ccf5c17b3a23e3c,2018-06-25 09:31:23,187.9000,39.8500
69544,9ec184faa31b594424cec063d4e4b4d4,1,ba230fc7bcaa2f503c46d73098caeb66,13d058e4eeac2ce8217660b2f8a05812,2018-07-03 03:17:54,229.9900,75.6900
54298,7b5d37ba26248e70f9b6c863d1964ed2,1,9b20cbe99cd2a76d917fd7c13cc75384,17e34d8224d27a541263c4c64b11a56b,2017-11-30 09:51:29,323.1800,27.5400
74661,aa0bc25a48a3d53cc009df13ed116b23,1,7b8c427eb09be88d32fcfaa8018f5caa,a416b6a846a11724393025641d4edd5e,2018-03-23 10:07:24,96.4900,11.3000


In [52]:
# Fix the weird BOM column name
category_translation = category_translation.rename(
    columns={"ï»¿product_category_name": "product_category_name"}
)

category_translation.columns

Index(['product_category_name', 'product_category_name_english'], dtype='object')

In [53]:
print("products columns:\n", products.columns.tolist())
print("\ncategory_translation columns:\n", category_translation.columns.tolist())

products columns:
 ['product_id', 'product_category_name', 'product_name_lenght', 'product_description_lenght', 'product_photos_qty', 'product_weight_g', 'product_length_cm', 'product_height_cm', 'product_width_cm']

category_translation columns:
 ['product_category_name', 'product_category_name_english']


In [54]:
# 1. Merge products with category translation to get English category names
products_merged = products.merge(
    category_translation,
    how="left",
    left_on="product_category_name",
    right_on="product_category_name"
)

# Rename translated column for clarity
products_merged = products_merged.rename(
    columns={"product_category_name_english": "category_english"}
)

products_merged[["product_id", "product_category_name", "category_english"]].head()

,product_id,product_category_name,category_english
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,perfumery
1,3aa071139cb16b67ca9e5dea641aaa2f,artes,art
2,96bd76ec8810374ed1b65e291975717f,esporte_lazer,sports_leisure
3,cef67bcfe19066a932b7673e239eb23d,bebes,baby
4,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas,housewares


In [55]:
# 2. Merge order_items with the enriched products table
items_products = order_items.merge(
    products_merged,
    how="left",
    on="product_id"
)

items_products.head()

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,category_english
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.9000,13.2900,cool_stuff,58.0000,598.0000,4.0000,650.0000,28.0000,9.0000,14.0000,cool_stuff
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.9000,19.9300,pet_shop,56.0000,239.0000,2.0000,"30,000.0000",50.0000,30.0000,40.0000,pet_shop
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.0000,17.8700,moveis_decoracao,59.0000,695.0000,2.0000,"3,050.0000",33.0000,13.0000,33.0000,furniture_decor
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.9900,12.7900,perfumaria,42.0000,480.0000,1.0000,200.0000,16.0000,10.0000,15.0000,perfumery
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.9000,18.1400,ferramentas_jardim,59.0000,409.0000,1.0000,"3,750.0000",35.0000,40.0000,30.0000,garden_tools


In [56]:
# 3. Merge with orders to bring in order-level info
items_products_orders = items_products.merge(
    orders,
    how="left",
    on="order_id"
)

items_products_orders.head()

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,category_english,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.9000,13.2900,cool_stuff,58.0000,598.0000,4.0000,650.0000,28.0000,9.0000,14.0000,cool_stuff,3ce436f183e68e07877b285a838db11a,delivered,2017-09-13 08:59:02,2017-09-13 09:45:35,2017-09-19 18:34:16,2017-09-20 23:43:48,2017-09-29 00:00:00
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.9000,19.9300,pet_shop,56.0000,239.0000,2.0000,"30,000.0000",50.0000,30.0000,40.0000,pet_shop,f6dd3ec061db4e3987629fe6b26e5cce,delivered,2017-04-26 10:53:06,2017-04-26 11:05:13,2017-05-04 14:35:00,2017-05-12 16:04:24,2017-05-15 00:00:00
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.0000,17.8700,moveis_decoracao,59.0000,695.0000,2.0000,"3,050.0000",33.0000,13.0000,33.0000,furniture_decor,6489ae5e4333f3693df5ad4372dab6d3,delivered,2018-01-14 14:33:31,2018-01-14 14:48:30,2018-01-16 12:36:48,2018-01-22 13:19:16,2018-02-05 00:00:00
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.9900,12.7900,perfumaria,42.0000,480.0000,1.0000,200.0000,16.0000,10.0000,15.0000,perfumery,d4eb9395c8c0431ee92fce09860c5a06,delivered,2018-08-08 10:00:35,2018-08-08 10:10:18,2018-08-10 13:28:00,2018-08-14 13:32:39,2018-08-20 00:00:00
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.9000,18.1400,ferramentas_jardim,59.0000,409.0000,1.0000,"3,750.0000",35.0000,40.0000,30.0000,garden_tools,58dbd0b2d70206bf40e62cd34e84d795,delivered,2017-02-04 13:57:51,2017-02-04 14:10:13,2017-02-16 09:46:09,2017-03-01 16:42:31,2017-03-17 00:00:00


In [57]:
# 4. Merge with sellers to add seller location
listings = items_products_orders.merge(
    sellers,
    how="left",
    on="seller_id"
)

listings.head()

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,category_english,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,seller_zip_code_prefix,seller_city,seller_state
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.9000,13.2900,cool_stuff,58.0000,598.0000,4.0000,650.0000,28.0000,9.0000,14.0000,cool_stuff,3ce436f183e68e07877b285a838db11a,delivered,2017-09-13 08:59:02,2017-09-13 09:45:35,2017-09-19 18:34:16,2017-09-20 23:43:48,2017-09-29 00:00:00,27277,volta redonda,SP
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.9000,19.9300,pet_shop,56.0000,239.0000,2.0000,"30,000.0000",50.0000,30.0000,40.0000,pet_shop,f6dd3ec061db4e3987629fe6b26e5cce,delivered,2017-04-26 10:53:06,2017-04-26 11:05:13,2017-05-04 14:35:00,2017-05-12 16:04:24,2017-05-15 00:00:00,3471,sao paulo,SP
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.0000,17.8700,moveis_decoracao,59.0000,695.0000,2.0000,"3,050.0000",33.0000,13.0000,33.0000,furniture_decor,6489ae5e4333f3693df5ad4372dab6d3,delivered,2018-01-14 14:33:31,2018-01-14 14:48:30,2018-01-16 12:36:48,2018-01-22 13:19:16,2018-02-05 00:00:00,37564,borda da mata,MG
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.9900,12.7900,perfumaria,42.0000,480.0000,1.0000,200.0000,16.0000,10.0000,15.0000,perfumery,d4eb9395c8c0431ee92fce09860c5a06,delivered,2018-08-08 10:00:35,2018-08-08 10:10:18,2018-08-10 13:28:00,2018-08-14 13:32:39,2018-08-20 00:00:00,14403,franca,SP
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.9000,18.1400,ferramentas_jardim,59.0000,409.0000,1.0000,"3,750.0000",35.0000,40.0000,30.0000,garden_tools,58dbd0b2d70206bf40e62cd34e84d795,delivered,2017-02-04 13:57:51,2017-02-04 14:10:13,2017-02-16 09:46:09,2017-03-01 16:42:31,2017-03-17 00:00:00,87900,loanda,PR


In [58]:
# Keep only delivered orders to avoid weird prices
if "order_status" in listings.columns:
    listings = listings[listings["order_status"] == "delivered"]

# Drop rows with missing key fields
key_cols = ["price", "product_id", "order_id"]
listings = listings.dropna(subset=key_cols)

# Quick shape check
print("Listings shape:", listings.shape)

# Peek at important columns
listings[[
    "order_id",
    "product_id",
    "price",
    "freight_value",
    "product_category_name",
    "category_english",
    "seller_id",
    "seller_city",
    "seller_state",
    "order_purchase_timestamp"
]].head()

Listings shape: (110197, 26)


,order_id,product_id,price,freight_value,product_category_name,category_english,seller_id,seller_city,seller_state,order_purchase_timestamp
0,00010242fe8c5a6d1ba2dd792cb16214,4244733e06e7ecb4970a6e2683c13e61,58.9000,13.2900,cool_stuff,cool_stuff,48436dade18ac8b2bce089ec2a041202,volta redonda,SP,2017-09-13 08:59:02
1,00018f77f2f0320c557190d7a144bdd3,e5f2d52b802189ee658865ca93d83a8f,239.9000,19.9300,pet_shop,pet_shop,dd7ddc04e1b6c2c614352b383efe2d36,sao paulo,SP,2017-04-26 10:53:06
2,000229ec398224ef6ca0657da4fc703e,c777355d18b72b67abbeef9df44fd0fd,199.0000,17.8700,moveis_decoracao,furniture_decor,5b51032eddd242adc84c38acab88f23d,borda da mata,MG,2018-01-14 14:33:31
3,00024acbcdf0a6daa1e931b038114c75,7634da152a4610f1595efa32f14722fc,12.9900,12.7900,perfumaria,perfumery,9d7a1d34a5052409006425275ba1c2b4,franca,SP,2018-08-08 10:00:35
4,00042b26cf59d7ce69dfabb4e55b4fd9,ac6c3623068f30de03045865e4e10089,199.9000,18.1400,ferramentas_jardim,garden_tools,df560393f3a51e74553ab94004ba5c87,loanda,PR,2017-02-04 13:57:51


In [61]:
import os

INTERIM_PATH = "../data/processed"
os.makedirs(INTERIM_PATH, exist_ok=True)

output_path = os.path.join(INTERIM_PATH, "cleaned_listings.csv")
listings.to_csv(output_path, index=False)

print("Saved cleaned listings to:", output_path)

Saved cleaned listings to: ../data/processed/cleaned_listings.csv


In [ ]:
import pandas as pd

listings = pd.read_csv("../data/processed/cleaned_listings.csv")
listings.head()

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,category_english,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,seller_zip_code_prefix,seller_city,seller_state
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.9000,13.2900,cool_stuff,58.0000,598.0000,4.0000,650.0000,28.0000,9.0000,14.0000,cool_stuff,3ce436f183e68e07877b285a838db11a,delivered,2017-09-13 08:59:02,2017-09-13 09:45:35,2017-09-19 18:34:16,2017-09-20 23:43:48,2017-09-29 00:00:00,27277,volta redonda,SP
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.9000,19.9300,pet_shop,56.0000,239.0000,2.0000,"30,000.0000",50.0000,30.0000,40.0000,pet_shop,f6dd3ec061db4e3987629fe6b26e5cce,delivered,2017-04-26 10:53:06,2017-04-26 11:05:13,2017-05-04 14:35:00,2017-05-12 16:04:24,2017-05-15 00:00:00,3471,sao paulo,SP
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.0000,17.8700,moveis_decoracao,59.0000,695.0000,2.0000,"3,050.0000",33.0000,13.0000,33.0000,furniture_decor,6489ae5e4333f3693df5ad4372dab6d3,delivered,2018-01-14 14:33:31,2018-01-14 14:48:30,2018-01-16 12:36:48,2018-01-22 13:19:16,2018-02-05 00:00:00,37564,borda da mata,MG
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.9900,12.7900,perfumaria,42.0000,480.0000,1.0000,200.0000,16.0000,10.0000,15.0000,perfumery,d4eb9395c8c0431ee92fce09860c5a06,delivered,2018-08-08 10:00:35,2018-08-08 10:10:18,2018-08-10 13:28:00,2018-08-14 13:32:39,2018-08-20 00:00:00,14403,franca,SP
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.9000,18.1400,ferramentas_jardim,59.0000,409.0000,1.0000,"3,750.0000",35.0000,40.0000,30.0000,garden_tools,58dbd0b2d70206bf40e62cd34e84d795,delivered,2017-02-04 13:57:51,2017-02-04 14:10:13,2017-02-16 09:46:09,2017-03-01 16:42:31,2017-03-17 00:00:00,87900,loanda,PR


### 3. Data Cleaning

#### 3.1 Handling Missing Values

#### 3.2 Category Parsing & Splitting ( cat1/ cat2/ cat3 )

#### 3.3 Outlier Detection

#### 3.4 Data Type fixes

### 4. Category Structure Analysis

#### 4.1 Distribution of Category Levels

#### 4.2 Category-Level Price Statistics

#### 4.3 Category Popularity & Product Counts

### 5 Price Analysis

#### 5.1 Overall Price Distribution

#### 5.2 Price by Category

#### 5.3 Price Transformation Check (Log Price, etc.)

### 6 Product Attribute EDA

#### 6.1 Brand Analysis

#### 6.2 Ratings & Reviews

#### 6.3 Attribute Correlations

### 7 Preliminary Business Insights

### 8 Export Cleaned Dataset